In [12]:
import operator
import functools

In [13]:
class Neuron:
    def __init__(self, inbound_neurons=[]):
        # Neurons from which this Node receives values
        self.inbound_neurons = inbound_neurons
        # Neurons to which this Node passes values
        self.outbound_neurons = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_neurons:
            n.outbound_neurons.append(self)

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_neurons` and
        store the result in self.value.
        """
        raise NotImplemented

In [20]:
class Input(Neuron):
    def __init__(self):
        Neuron.__init__(self)

    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value

In [28]:
class Add(Neuron):
    def __init__(self, *inputs):
        Neuron.__init__(self, [*inputs])

    def forward(self, value=None):
        self.value = sum([n.value for n in self.inbound_neurons])

In [29]:
class Mul(Neuron):
    def __init__(self, *inputs):
        Neuron.__init__(self, [*inputs])

    def forward(self, value=None):
        self.value = functools.reduce(operator.mul, [n.value for n in self.inbound_neurons], 1)

In [73]:
class Linear(Neuron):
    def __init__(self, inputs, weights, bias):
        Neuron.__init__(self, inputs)
        self.weights = weights
        self.bias = bias
        
    def forward(self):
        xi = [n.value for n in self.inbound_neurons]
        wi = [w.value for w in self.weights]
        self.value = sum([a*b for a,b in zip(xi, wi)])+self.bias.value
        
#    def forward(self):
#        self.value = self.bias.value
#        for w, x in zip(self.weights, self.inbound_neurons):
#            self.value += w.value * x.value

In [69]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_neurons = [n for n in feed_dict.keys()]

    G = {}
    neurons = [n for n in input_neurons]
    while len(neurons) > 0:
        n = neurons.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_neurons:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            neurons.append(m)

    L = []
    S = set(input_neurons)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_neurons:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [70]:
def forward_pass(output_neuron, sorted_neurons):
    """
    Performs a forward pass through a list of sorted neurons.

    Arguments:

        `output_neuron`: A neuron in the graph, should be the output neuron (have no outgoing edges).
        `sorted_neurons`: a topologically sorted list of neurons.

    Returns the output neuron's value
    """

    for n in sorted_neurons:
        n.forward()

    return output_neuron.value

In [71]:
x, y, z = Input(), Input(), Input()

f = Add(x, y, z)
feed_dict = {x: 4, y: 5, z: 10}
graph = topological_sort(feed_dict)
output = forward_pass(f, graph)
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))

f = Mul(x, y, z)
feed_dict = {x: 4, y: 5, z: 10}
graph = topological_sort(feed_dict)
output = forward_pass(f, graph)
print("{} * {} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))

4 + 5 + 10 = 19 (according to miniflow)
4 * 5 * 10 = 200 (according to miniflow)


In [72]:
inputs = [x, y, z]
weight_x, weight_y, weight_z = Input(), Input(), Input()
weights = [weight_x, weight_y, weight_z]
bias = Input()
f = Linear(inputs, weights, bias)
feed_dict = {
    x: 6,
    y: 14,
    z: 3,
    weight_x: 0.5,
    weight_y: 0.25,
    weight_z: 1.4,
    bias: 2
}
graph = topological_sort(feed_dict)
output = forward_pass(f, graph)
print(output)

12.7
